In [4]:
from chromo.models import Pythia8
from matplotlib import pyplot
import boost_histogram as bh
import numba as nb
import numpy as np

def plot(hists, xlim = None, ylim = None, xlog = False, ylog = False, 
         xlabel = None, ylabel = None, norm = False, fnc = None):
    for hist in hists:
        if norm: hist.normalize()
        x = [x for x0, x1 in zip(
            hist.getBinEdges()[::1], hist.getBinEdges()[1::1])
            for x in (x0, x1)]
        y = [y for y0 in hist.getBinContents()
             for y in (y0, y0)]
        pyplot.plot(x, y, label = hist.getTitle())
    if xlog: pyplot.xscale("log")
    if ylog: pyplot.yscale("log")
    if xlim: pyplot.xlim(xlim)
    if ylim: pyplot.ylim(ylim)
    if xlabel: pyplot.xlabel(xlabel)
    if ylabel: pyplot.ylabel(ylabel)
    if fnc: fnc()
    pyplot.legend()
    pyplot.show()

In [ ]:
class Thrust:
    def __init__(self, select):
        self.select = select
        self.NSTUDYMIN = 2

    def analyze(self, event):
        fs = event.final_state()

        if self.select > 2:
            fs = fs[fs.charge != 0]
        elif self.select == 2:
            # deselect !isVisible
            pass

        nStudy = len(fs)
        ptot = fs.p_tot
        pOrder = np.transpose([fs.px, fs.py, fs.pz, ptot])
        pSum = np.sum(pOrder, axis=0)

        if nStudy < self.NSTUDYMIN:
            return False

        def cross3(p1, p2):
            np.cross(p1[:3], p2[:3])

        # Try all combinations of reference vector orthogonal to two particles.
        for i1 in range(nStudy - 1):
            for i2 in range(i1 + 1, nStudy):
                nRef = cross3( pOrder[i1], pOrder[i2])
                nRef /= np.linalg.norm(nRef)
                pPart = 0.

                # Add all momenta with sign two choices for each reference particle.
                for (int i = 0 i < nStudy ++i) if (i != i1 && i != i2) {
                if (dot3(pOrder[i], nRef) > 0.) pPart += pOrder[i]
                else                            pPart -= pOrder[i]
                }
                for (int j = 0 j < 4 ++j) {
                if      (j == 0) pFull = pPart + pOrder[i1] + pOrder[i2]
                else if (j == 1) pFull = pPart + pOrder[i1] - pOrder[i2]
                else if (j == 2) pFull = pPart - pOrder[i1] + pOrder[i2]
                else             pFull = pPart - pOrder[i1] - pOrder[i2]
                pFull.e(pFull.pAbs())
                if (pFull.e() > pMax.e()) pMax = pFull

        // Maximum gives thrust axis and value.
        eVal1 = pMax.e() / pSum.e()
        eVec1 = pMax / pMax.e()
        eVec1.e(0.)

        // Subtract momentum along thrust axis.
        double pAbsSum = 0.
        for (int i = 0 i < nStudy ++i) {
            pOrder[i] -= dot3( eVec1, pOrder[i]) * eVec1
            pOrder[i].e(pOrder[i].pAbs())
            pAbsSum += pOrder[i].e()
        }

        // Simpleminded major and minor axes if too little transverse left.
        if (pAbsSum < MAJORMIN * pSum.e()) {
            if ( abs(eVec1.pz()) > 0.5) eVec2 = Vec4( 1., 0., 0., 0.)
            else                        eVec2 = Vec4( 0., 0., 1., 0.)
            eVec2 -= dot3( eVec1, eVec2) * eVec1
            eVec2 /= eVec2.pAbs()
            eVec3  = cross3( eVec1, eVec2)
            return true
        }

        // Try all reference vectors orthogonal to one particles.
        pMax = 0.
        for (int i1 = 0 i1 < nStudy ++i1) {
            nRef = cross3( pOrder[i1], eVec1)
            nRef /= nRef.pAbs()
            pPart = 0.

            // Add all momenta with sign two choices for each reference particle.
            for (int i = 0 i < nStudy ++i) if (i != i1) {
            if (dot3(pOrder[i], nRef) > 0.) pPart += pOrder[i]
            else                            pPart -= pOrder[i]
            }
            pFull = pPart + pOrder[i1]
            pFull.e(pFull.pAbs())
            if (pFull.e() > pMax.e()) pMax = pFull
            pFull = pPart - pOrder[i1]
            pFull.e(pFull.pAbs())
            if (pFull.e() > pMax.e()) pMax = pFull
        }

        // Maximum gives major axis and value.
        eVal2 = pMax.e() / pSum.e()
        eVec2 = pMax / pMax.e()
        eVec2.e(0.)

        // Orthogonal direction gives minor axis, and from there value.
        eVec3 = cross3( eVec1, eVec2)
        pAbsSum = 0.
        for (int i = 0 i < nStudy ++i)
            pAbsSum += abs( dot3(eVec3, pOrder[i]) )
        eVal3 = pAbsSum / pSum.e()

        // Done.
        return true

        }


    def thrust(self):
        return 0


class AnalyzeThrust:
    """
    Class to perform a thrust analysis.
    We write this as a class, so we can use the Pythia parallel framework.
    """
    
    def __init__(self, title = "default", nBins = 20, xLow = 0, xHigh = 0.5):
        """
        Define the constructor.

        title: title of the histogram.
        nBins: number of bins in the histogram.
        xLow:  lowest bin edge of the histogram.
        xHigh: highest bin edge of the histogram.
        """
        # Create the histogram for storing the thrust.
        self.hist = bh.Histogram(nBins, xLow, xHigh)

        # Create the thrust analysis object. The argument is as follows.
        # 1: all final-state particles
        # 2: all observable final-state particles.
        # 3: only charged final-state particles.
        self.thrust = Thrust(2)
        
    def run(self, event):
        """
        Define the analysis. Note, we are working with a single histogram,
        so we need to make sure that the setting
        Parallelism:processAsync is alsways off.
        
        pythiaNow: the Pythia instance containing the event to analyze.
        """
        # Calculate the thrust and fill the histogram.
        self.thrust.analyze(event)
        self.hist.fill(1 - self.thrust.thrust())